Vorbereitung der Umgebung
Benötigte Packete
-nympy
-matplotlib
-gstools

-jupyter-notebook Umgebung (z.B. jupyterlab)

- jupyterlab mit "jupyter-lab" in Konsole ausführen



In [ ]:
#Importiere benötgte Pakete
import numpy as np
from matplotlib import pyplot as plt
import gstools as gs
import lhsmdu as lhs #<- eventuell nachinstallieren, war nicht in der mail aufgelistet



In [ ]:
#Importiere alle für Hand-Ons benötigten Funktionen
from ml_kriging import calculateKrigingModel, plotKrigModel, plotCovarianceModel, getSEmapOfKrigModel, getBestCovarianceModel, plotFunction2D
from ml_kriging import getPointWithMaxValue, addOnePoint, getMSE, analyseFunction 
from ml_kriging import addRandomPoints, addEdgePoints, getRandomPoints, getGridPoints, getCenteredGridPoints, getCenteredRandomGridPoints, getCenteredRandomGridCombinedPoints, getCenteredRandomGridCombinedPointsWithEdges, getInitPoints, getLHSPoints, getMCPoints
from ml_kriging import f1, f2, f3, f_herbie_1, f_herbie_2, f_herbie_3, f_herbie_4, f_herbie_complex, f_rosenbrock, f_zum_versagen_von_kriging, f_shubert_1_not_for_hand_ons, f_shubert_2_not_for_hand_ons, f_parabolid, f_hyperbolic_parabolid, f_eggholder, f_six_hump_camel, f_three_hump_camel, f_branin  


#####################################################
##############  HANDS-ON: Einleitung   ##############
#####################################################

# Erklärung der vorhanden Funktionen

distribution_type
"RandomPoints" (bzw. "else-case),           #zufällige verteilung
"GridPoints"                                #im gitter angeordnet mit randpunkten
"CenteredGridPoints"                        #im gitter angeordnet, ohne randpunkte
"CenteredRandomGridPoints"                  #in gitter aufgeteilt und im quadranten eine zufallszahl gewählt
"CenteredRandomGridCombinedPoints"          #kombination von random-grid und random punkten
"CenteredRandomGridCombinedPointsWithEdges" #kombination auf random-grid und random punkten +4 eckpunkte
"LHS"                                       #latin hypercube sampling (LHS)
"MC"                                        #monte carlo
x_p,y_p = getInitPoints(distribution_type, n_datapoints, min_x,max_x, min_y,max_y)

In dieser Funktion werden die einzelnen Funktionen aufgerufen: 
- addRandomPoints
- addEdgePoints
- getRandomPoints
- getGridPoints
- getCenteredGridPoints
- getCenteredRandomGridPoints
- getCenteredRandomGridCombinedPoints
- getCenteredRandomGridCombinedPointsWithEdges
- getLHSPoints
- getMCPoints
- - - - - - - - -     
plotFunction2D(f, ax) #plotted die in handons verwendete funktionen
- - - - - - - - -   
kriging_type
"simple"
"ordinary"
"universal_linear"
"universal_quadratic"
calculateKrigingModel(kriging_type, model_in, x_in, y_in, z_in) 
- - - - - - - - -   
plotKrigModel #plotet das kriging-model
- - - - - - - - -   
plotCovarianceModel #plotet das kovarianzmodel
- - - - - - - - -   
getSEmapOfKrigModel #berechnet quadratische abweichung von wahrer funktion
- - - - - - - - -   

        "Gaussian": gs.Gaussian,
        "Exponential": gs.Exponential,
        "Matern": gs.Matern,
        "Stable": gs.Stable,
        "Rational": gs.Rational,
        "Circular": gs.Circular,
        "Spherical": gs.Spherical,
        "SuperSpherical": gs.SuperSpherical
getBestCovarianceModel(x_p, y_p, z_p, textoutput = False, models) #fittet die übergebenen Kovarianzmodelle und gibt das model mit dem besten pseudo-r2 wert zurück
- - - - - - - - -   
getPointWithMaxValue #gibt den punkt x,y mit dem größten erwarteten Krigingmodelfehler zurück
- - - - - - - - -   
addOnePoint #fügt einen punkt zum bestehen array
- - - - - - - - -   
getMSE #berechnet mean squared error von einem array
- - - - - - - - - - - -
Mit dieser Funktion kann der MSE in Abhängigkeit von der Anzahl der Startpunkte untersuchen werden.
Man kann in diesem notebook definierte Funktion damit untersuchen
Die Logik führt zum Einen den Fit mit allen Punkten durch, zum Anderen wird zusätzlich mit Aufsplittung in Startpunkte + größter SE das Model laufend verbessert
analyseFunction(f1, n_points_to_fit, n_runs, init_fraction, fig, ax1, ax2, ax3, ax4)
- - - - - - - - - - - -
f1, f2, f3, .... sind die definierten Funktionen als Klassen, die in Handons verwendet werden, mit
 .getValue(x,y) zur Rückgab des funktionwertes
 .getRange() zur Rückgabe des Wertebereichs

Liste der Funktionen:
- f1
- f2
- f3
- f_herbie_1
- f_herbie_2
- f_herbie_3
- f_herbie_4
- f_rosenbrock
- f_zum_versagen_von_kriging
- f_parabolid
- f_hyperbolic_parabolid
- f_eggholder
- f_six_hump_camel
- f_three_hump_camel
- f_branin 

Funktionsklasse
#Erstelle deine eigne funktion, oder verwende eine der bereits definierten
    class f_deine_eigene_funktion:
        def __init__(self):
            #definiere Wertebereich der Funktion
            self.min_x = -1
            self.max_x = 1
            self.min_y = -1
            self.max_y = 1

        def getRange(self):
            return self.min_x, self.max_x, self.min_y, self.max_y

        def getValue(self, x, y):
            #definiere funktion
            #z.b. return x + y (geneigte platte)
            #z.b. return x**2 + y**2 Paraboloid
            return x**2-y**2
            
Weitere Funktionen: https://www.sfu.ca/~ssurjano/optimization.html

# ##################################################################
# # HANDS-ON: Einleitung (Theorie -> Praxisbeispiel)  ##############
# ##################################################################

In [ ]:
#wähle die funktion aus
f_current = f_hyperbolic_parabolid()

#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_current.getRange()

fig, ax = plt.subplots()
plotFunction2D(f_current, ax)

# Erklärung Stützpunkte

In [ ]:
fig, ax = plt.subplots()
ax.title.set_text('Auswahl Stützpunkte')

#definiere parameter
n_datapoints = 64
#distribution_type
# / "MC"
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
# / "LHS"
x_p,y_p = getInitPoints("GridPoints", n_datapoints, min_x,max_x, min_y,max_y)


plt.xlim(min_x, max_x)
plt.ylim(min_y, max_y)
ax.scatter(x_p,y_p, color = "red")


# Erklärung Variogramm

In [ ]:
#definiere parameter
n_known_datapoints = 50
n_points_to_plot = 100

#distribution_type
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
x_p,y_p = getInitPoints("LHS", n_known_datapoints, min_x,max_x, min_y,max_y)

#berechne funktionswerte
z_p = f_current.getValue(x_p,y_p)

fig, ax = plt.subplots()

bin_center, gamma = gs.vario_estimate((x_p,y_p), z_p)
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

models = {
    "Gaussian": gs.Gaussian,
    #"Exponential": gs.Exponential,
    "Matern": gs.Matern,
    "Stable": gs.Stable,
    #"Rational": gs.Rational,
    #"Circular": gs.Circular,
    #"Spherical": gs.Spherical,
    #"SuperSpherical": gs.SuperSpherical,
    #"JBessel": gs.JBessel, #keine Varianzberechnung möglich !!
}


# plot the estimated variogram
ax.scatter(bin_center, gamma, color="k", label="data")


# fit all models to the estimated variogram
scores = {}
max_distance_variogram = 0.75*max(max_x-min_x, max_y-min_y)
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=False)
    fit_model.plot(x_max=max_distance_variogram, ax=ax)
    scores[model] = r2

ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
print("RANKING by Pseudo-r2 score")
best_model = {}
best_score = -1
for i, (model, score) in enumerate(ranking, 1):
    if score > best_score:
        best_score = score
        best_model = models[model](dim=2)
    print(f"{i:>6}. {model:>15}: {score:.5}")




# Erklärung Kriging-Model(s)

In [ ]:
#bereite punkte zum plotten 
x,y = np.meshgrid(np.linspace(min_x,max_x,n_points_to_plot),np.linspace(min_y,max_y,n_points_to_plot))
z = f_current.getValue(x, y)
z_arr1d = np.reshape(z, n_points_to_plot**2)

#berechne kriging-model
#"simple"
#"ordinary"
#"universal_linear"
#"universal_quadratic"
krig = calculateKrigingModel("universal_linear", best_model, x_p,y_p, z_p)

fig, (ax1, ax2) = plt.subplots(1,2)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
fig.set_size_inches(15,5)


-----------------------------------------------------------------------------
# -----------------------------------------------------------
# HANDS-ON: Variogramme (=Kovarianz, Varianz, Semivariogramm) erstellen und verstehen
# -----------------------------------------------------------
-----------------------------------------------------------------------------

# HANDS-ON: (Semi-)Variogrammaufgabe 1 

In [ ]:
#Erstelle deine eigene funktion, oder verwende eine der bereits definierten
class f_deine_eigene_funktion:
    def __init__(self):
        #definiere Wertebereich der Funktion
        self.min_x = 0
        self.max_x = 1
        self.min_y = 0
        self.max_y = 1

    def getRange(self):
        return self.min_x, self.max_x, self.min_y, self.max_y

    def getValue(self, x, y):
        #definiere funktion
        return x + y

#wähle die funktion aus
f_bekannte_funktion = f_deine_eigene_funktion()    #wenn du die von dir definierte funktion verwenden möchtest
#f_bekannte_funktion = f2()                         #wenn du z.b. die hier implementierte f2

#funktion zum anschauen der funktion
fig, ax = plt.subplots()
plotFunction2D(f_bekannte_funktion, ax)

In [ ]:
#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_bekannte_funktion.getRange()

##############################################################
######## hier gerne mit Parametern   rumspielen ##############
##############################################################

#fitte variogram (bei dieser Aufgabe gerne mehr Punkte verwenden, es geht ums Verständnis der Variogramme
#startpunkte
n_known_datapoints = 100

#distribution_type
# / "RandomPoints" (bzw. "else-case),
# / "GridPoints"
# / "CenteredGridPoints"
# / "CenteredRandomGridPoints"
# / "CenteredRandomGridCombinedPoints"
# / "CenteredRandomGridCombinedPointsWithEdges"
x_p,y_p = getInitPoints("LHS", n_known_datapoints, min_x,max_x, min_y,max_y)
z_p = f_bekannte_funktion.getValue(x_p,y_p)

######## das spiel ist vorbei ##############

#berechne experimentelles variogram

bin_center, gamma = gs.vario_estimate((x_p, y_p), z_p)
#enferne fehlende distanzen
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

#schaue das ergebnis an

fig, ax = plt.subplots()
# plot the estimated variogram
ax.scatter(bin_center, gamma, color="k", label="data")

Welche fit-funktion würde sich hier anbieten ? (den Verlauf in etwa, ich weiß selber nicht wie die alle aussehen ;) )

In [ ]:
#wähle deine favoriten aus

##############################################################
######## hier gerne mit Cov-modellen rumspielen ##############
##############################################################

models = {
    "Gaussian": gs.Gaussian,
    #"Exponential": gs.Exponential,
    #"Matern": gs.Matern,
    #"Stable": gs.Stable,
    #"Rational": gs.Rational,
    #"Circular": gs.Circular,
    #"Spherical": gs.Spherical,
    #"SuperSpherical": gs.SuperSpherical,
    #"JBessel": gs.JBessel, #keine Varianzberechnung möglich !!
}

######## das spiel ist vorbei ##############

fig, ax = plt.subplots()
#betrachte nun das gefittete covarianz-modell
# plot the estimated variogram
ax.scatter(bin_center, gamma, color="k", label="data")


# fit all models to the estimated variogram
scores = {}
max_distance_variogram = 0.75*max(max_x-min_x, max_y-min_y)
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=False)
    fit_model.plot(x_max=max_distance_variogram, ax=ax)
    scores[model] = r2

ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
print("RANKING by Pseudo-r2 score")
best_model = {}
best_score = -1
for i, (model, score) in enumerate(ranking, 1):
    if score > best_score:
        best_score = score
        best_model = models[model](dim=2)
    print(f"{i:>6}. {model:>15}: {score:.5}")

# Aufgabe:
- Gehe zum vorherigen Punkt, reduziere die Anzahl der Stützpunkte (n_known_datapoints) und wiederhole die Schritte.
- Welche auswirkungen hat es auf variogram ?
- Ab welcher anzahl der Punkte würdest du keinen zuverlässigen Fit der Kovarianz-funktion erwarten?

# HANDS-ON: (Semi-)Variogrammaufgabe 2 

In [ ]:
#Führe den gesamten Block aus

######## das spiel ist vorbei ##############

f_var_2 = f_shubert_1_not_for_hand_ons()
min_x, max_x, min_y, max_y = f_var_2.getRange()
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPoints", 3000, min_x,max_x, min_y,max_y)
z_p = f_var_2.getValue(x_p,y_p)

fig, (ax1, ax2) = plt.subplots(1,2)

bins = np.linspace(0,3,40)
bin_center, gamma = gs.vario_estimate((x_p,y_p), z_p, bin_edges=bins)
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

models = {
    "JBessel": gs.JBessel,
}


# plot the estimated variogram
ax2.scatter(bin_center, gamma, color="k", label="data")

# fit all models to the estimated variogram
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=False)
    fit_model.plot(x_max=5, ax=ax2)

plotFunction2D(f_var_2, ax1)
fig.set_size_inches(15,5)

# Aufgabe:
Im Plot links siehst du die wahre Funktion, rechts das entsprechende Variogram.
- Überlegt es euch, warum das Variogram so aussieht, wie es aussieht?
- Welche Bedeutung haben insbesondere Abstände zwischen den Schwingungen?

# HANDS-ON: Varianzaufgabe 3

In [ ]:
#Führe den gesamten Block aus

######## das spiel ist vorbei ##############

f_var_3 = f_shubert_2_not_for_hand_ons()
min_x, max_x, min_y, max_y = f_var_3.getRange()
x_p,y_p = getInitPoints("CenteredRandomGridCombinedPoints", 3000, min_x,max_x, min_y,max_y)
z_p = f_var_3.getValue(x_p,y_p)
fig, (ax1, ax2) = plt.subplots(1,2)

bins = np.linspace(0,3,40)
bin_center, gamma = gs.vario_estimate((x_p,y_p), z_p, bin_edges=bins)
bin_center = bin_center[gamma > 0]
gamma = gamma[gamma > 0]

models = {
    "JBessel": gs.JBessel,
}


# plot the estimated variogram
ax2.scatter(bin_center, gamma, color="k", label="data")

# fit all models to the estimated variogram
for model in models:
    fit_model = models[model](dim=2)
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True, nugget=True)
    fit_model.plot(x_max=5, ax=ax2)

plotFunction2D(f_var_3, ax1)

fig.set_size_inches(15,5)

# Aufgabe:
- Hat sich das Variogram geändert ?
- Falls ja, was könnte diese Änderung aussagen ?

# --------------------------------------------------------------------
# HANDS-ON: Kriging-Modelle erstellen und verstehen 
# --------------------------------------------------------------------

# HANDS-ON: Kriging 1, Manuelle Auswahl der Punkte

In [ ]:
#Erstelle deine eigene funktion, oder verwende eine der bereits definierten
class f_deine_eigene_funktion_2:
    def __init__(self):
        #definiere Wertebereich der Funktion
        self.min_x = 0
        self.max_x = 1
        self.min_y = 0
        self.max_y = 1

    def getRange(self):
        return self.min_x, self.max_x, self.min_y, self.max_y

    def getValue(self, x, y):
        #definiere funktion
        return x + y

#wähle die funktion aus
#f_bekannte_funktion_2 = f_deine_eigene_funktion_2()    #wenn du die von dir definierte funktion verwenden möchtest
#f_bekannte_funktion_2 = f_herbie_4()                         #wenn du z.b. die hier implementierte f_herbie_3
f_bekannte_funktion_2 = f_eggholder()                      #wenn du z.b. die hier implementierte f_eggholder() : <- durchaus interessant für diese aufgabe, da relativ komplex

#funktion zum anschauen der funktion
fig, ax = plt.subplots()
plotFunction2D(f_bekannte_funktion_2, ax)

In [ ]:
#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_bekannte_funktion_2.getRange()


##############################################################
######## hier gerne mit Parametern und Punkten rumspielen ###
##############################################################

#lege startpunkte manuell / visuell fest
#entscheide dich für eine bestimmte anzahl "n_max_points" der startpunkte (tipp: nicht weniger als 15, je nach fkt-komplexität)

#wird im nächsten Abschnitt verwendet, bitte manuell diese Anzahl an Punkten auswählen und nicht schummeln
n_max_points = 30 # <- Anpassen

x_p = []
y_p = []
#beispiel zum hinzufügen von vier eckpunkten punkten (x,y)
#für wertebereich 0-1
#x_p , y_p = addOnePoint(n_new, y_new, x_p , y_p)

# "n_max_points"mal die funktion verwenden
#x_p , y_p = addOnePoint(0.0, 0.0, x_p , y_p)
#x_p , y_p = addOnePoint(0.0, 1.0, x_p , y_p)
#x_p , y_p = addOnePoint(1.0, 0.0, x_p , y_p)
#x_p , y_p = addOnePoint(1.0, 1.0, x_p , y_p)
# ............
#x_p , y_p = addOnePoint(1.1, 2.0, x_p , y_p)


######## das spiel ist vorbei ##############

if len(x_p) > 0:

    #berechne die funktionswerte für startpunkte
    z_p = f_bekannte_funktion_2.getValue(x_p, y_p)

    #fitte variogram manuell (aus der variogram-aufgabe copy-pasten) oder automatisch mit der folgenden funktion
    best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

    #falls ihr das variogram anschauen wollt, auskommentieren
    #fig, ax1 = plt.subplots()
    #plotCovarianceModel(bin_center, gamma, best_model, ax1)

    #berechne kriging-model
    
    ##############################################################
    ######## hier gerne mit modellen rumspielen ###
    ##############################################################
    krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p)
    ######## das spiel ist vorbei ##############

    #man kann das feld und sqrt(varianz) = std.abw anschauen
    fig, (ax1, ax2) = plt.subplots(1,2)
    plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
    fig.set_size_inches(15,5)

    #Berechnete quadratische Abweichung zur wahren Funktion
    fig, ax1 = plt.subplots()
    se_map, mse_manuell = getSEmapOfKrigModel(krig, f_bekannte_funktion_2, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1)

    print("MSE manuelle Datenpunkteauswahl: %.5f"%mse_manuell)
else:
    print("Ihr habt keine Stützpunkte ausgewählt")

# HANDS-ON: Kriging 2, Auswahl der Punkte mithilfe von max. erwarteten Modelfehler

In [ ]:
################# Die zu untersuchende Funktion bleibt aus manuellem Abschnitt         ##########

In [ ]:
#man muss mit bestimmten Anteil der Punkte starten (lass es mindestens 10 sein)
#die restlichen Punkte (gleiche Anzahl wie bei der manuellen Selektion) werden einzeln hinzugefügt

##############################################################
######## hier gerne mit Startpunkten spielen #################
##############################################################
n_start_points = 20 #<- kleiner als "n_max_points" setzen

######## das spiel ist vorbei ##############

x_p,y_p = getInitPoints("LHS", n_start_points, min_x,max_x, min_y,max_y) #<- kann man rumspielen, muss man nicht
z_p = f_bekannte_funktion_2.getValue(x_p, y_p)
best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p) #<- kann man rumspielen, muss man nicht
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
plotCovarianceModel(bin_center, gamma, best_model, ax3)
fig.set_size_inches(25,5)


########## begin der zu wiederholende teil, bis alle Punkte hingefügt sind ##########
for i in range(n_max_points - n_start_points):
    #hinfüge den punkt mit dem größten Modelfehler (manuell/visuell/glaskugel oder mit funktion)
    x_new, y_new = getPointWithMaxValue(krig, min_x, max_x, min_y, max_y)
    z_new = f_bekannte_funktion_2.getValue(x_new, y_new)
    x_p = np.append(x_p, x_new)
    y_p = np.append(y_p, y_new)
    z_p = np.append(z_p, z_new)

    best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

    #berechne kriging-model
    krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p) #<- sollte gleiche wie oben ausgewählt sein

    #man kann das feld und sqrt(varianz) = std.abw anschauen
    fig, (ax1, ax2, ax3) = plt.subplots(1,3)
    plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
    plotCovarianceModel(bin_center, gamma, best_model, ax3)
    ax1.scatter(x_new,y_new, color = "white", s=200)
    ax2.scatter(x_new,y_new, color = "white", s=200)
    fig.set_size_inches(25,5)
    
#im plot sind die neu hinzugefügten Punkte mit weiß gekennzeichnet

#Quadtrierte Abweichung von der wahren Funktion
fig, ax1 = plt.subplots()
se_map, mse_auto = getSEmapOfKrigModel(krig, f_bekannte_funktion_2, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1)
fig.set_size_inches(10,7)

print("MSE manuelle Datenpunkteauswahl: %.5f"%mse_manuell)
print("MSE Datenpunkteauswahl nach größtem Modelfehler: %.5f"%mse_auto)

if mse_manuell < mse_auto:
    print("\nGlückwusch !! Ihr wart bei der Selektion der Stützpunkte besser, als automatisches Hinfügen nach großtem Modelfehler !!\n")



# Aufgabe:
- Wird mit jedem hinzugefügten Punkt das Variogram besser ?
- Wird mit jedem hinzugefügten Punkt die vorhersagte Funktion besser ?
- Habt ihr die wichtigen Stützpunkte besser geschätzt ? (Ist der Fehler im ersten Fall kleiner)

# HANDS-ON: Eine unbekannte Funktion mit minimaler Anzahl der Stützpunkten nähern 

In [ ]:
##############################################################
######## hier gerne mit Startpunkten spielen #################
##############################################################

#wähle die funktion aus (eine, die ihr noch nicht angeschaut habt)
#f_unbekannte_funktion = f1()                              # kann man nehmen, muss man nicht
#f_unbekannte_funktion = f_branin()                        # kann man nehmen, ist OK <- eher einfache funktion
#f_unbekannte_funktion = f_six_hump_camel()                # ist i.O. für diese aufgabe
f_unbekannte_funktion = f_eggholder()                      # durchaus interessant für diese aufgabe, da relativ komplex, falls in der vorherigen aufgabe nicht genommen

######## das spiel ist vorbei ##############

#extrahiere wertebereich
min_x, max_x, min_y, max_y = f_unbekannte_funktion.getRange()

n_start_points = 15
x_p,y_p = getInitPoints("LHS", n_start_points, min_x,max_x, min_y,max_y) #<- kann man rumspielen, muss man nicht

#berechne die funktionswerte für startpunkte
z_p = f_unbekannte_funktion.getValue(x_p, y_p)

#fitte variogram manuell (aus der variogram-aufgabe copy-pasten) oder automatisch mit der folgenden funktion
best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)
#falls ihr das variogram anschauen wollt, auskommentieren
#fig, ax1 = plt.subplots()
#plotCovarianceModel(bin_center, gamma, best_model, ax1)

#berechne kriging-model
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p) #<- kann man rumspielen

#man kann das feld, sqrt(varianz) = std.abw  und variogram anschauen
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
plotCovarianceModel(bin_center, gamma, best_model, ax3)
fig.set_size_inches(20,5)


In [ ]:
#das ist die Ausgangszustand. Versucht mit möglichts wenigen zusätzlichen Punkten bestmögliche Beschreibung der Funktion erhalten 
#(die ihr eigentlich nicht kennt)

###### diesen schritt mehrfach ausführen (bzw. kompletten code-block copy-pasten und in eine neue code-zeile einfügen, um vorherigen zustand zu speichern), bis ihr mitm ergebniss zufrieden seid

# hinfüge den punkt mit dem größten Modelfehler oder den manuellen punkt, der ihr glaubt würde den fit verbessern

#hier mit dem größten SE
x_new, y_new = getPointWithMaxValue(krig, min_x, max_x, min_y, max_y) #oder manuell festlegen
x_p , y_p = addOnePoint(x_new, y_new, x_p , y_p)
#hier manuell (auskommentieren, die obere funktion kommentieren)
#x_p , y_p = addOnePoint(0.0, 0.0, x_p , y_p) #<- hier kann man rumspielen

z_new = f_unbekannte_funktion.getValue(x_new, y_new)
z_p = np.append(z_p, z_new)

best_model, best_score, bin_center, gamma = getBestCovarianceModel(x_p, y_p, z_p, True)

# berechne kriging-model
krig = calculateKrigingModel("universal_quadratic", best_model, x_p, y_p, z_p) #<- kann man rumspielen

# man kann das feld und sqrt(varianz) = std.abw anschauen
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
plotKrigModel(krig, min_x, max_x, min_y, max_y, x_p, y_p, fig, ax1, ax2)
plotCovarianceModel(bin_center, gamma, best_model, ax3)
ax1.scatter(x_new,y_new, color = "white", s=200)
ax2.scatter(x_new,y_new, color = "white", s=200)
fig.set_size_inches(20, 5)

print("Anzahl der Punkte im Model: %d"%len(z_p))

# Aufgabe:
- Mit wie vielen Punkten konntet ihr die "wahre" Funktion erahnen?
- Diese Teilaufgabe kann mit anderen Funktionen, wenn ihr Lust habt, wiederholt werden
- Diese Teilaufgabe kann mit der anderen Krigingmethode ("simple","ordinary","universal_linear","universal_quadratic") wiederholt werden

# HANDS-ON: Signifikanz der Modelgenauigkeit vs. Datenpunkteanzahl

Euch steht die Funktion analyseFunction(f1, n_points_to_fit, n_runs, init_fraction, fig, ax1, ax2, ax3, ax4) zur Verfügung,
Mit dieser Funktion kann der MSE in Abhängigkeit von der Anzahl der Startpunkte untersuchen werden.
Man kann in diesem notebook definierte Funktion damit untersuchen
Die Logik führt zum Einen den Fit mit allen Punkten durch, zum Anderen wird zusätzlich mit Aufsplittung in Startpunkte + größter SE das Model laufend verbessert


In [ ]:
##############################################################
######## tobbt euch aus  #####################################
##############################################################

#wähle die Funktion zur statistischen Analyse aus
f_funktion_zum_analysieren = f_herbie_3()

n_points_to_fit = [15, 20, 25, 30] #anzahl der stützpunkte
n_runs = 10 #anzahl der wiederholungen mit gleichen parametern (übertreibt nicht)
init_faction = 0.7 #anteil an startpunkten zum ersten berechnen des models, der rest wird nach dem größten SE hinzugefügt
krig_type = "universal_quadratic" #("simple","ordinary","universal_linear","universal_quadratic")
start_points = "LHS" # ist wirklich LHS besser als CenteredRandomGridCombinedPointsWithEdges ?

######## schluss damit  #####################################

#Laufzeit mit den nicht veränderten Parametern ca. 5 min
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
model_mse, model_mse_std, best_krige, best_cov_model, best_bin_center, best_gamma = analyseFunction(f_funktion_zum_analysieren,n_points_to_fit, n_runs, init_faction, fig, ax1, ax2, ax3, ax4, krig_type, start_points)

#die Funktion "analyseFunction" plottet das best gefundene Modell (wird wohl das, mit den meisten Punkten sein)


# Aufgabe:
- War das Ergebnis zu erwarten? Eklärbar ?
- Diese Teilaufgabe kann mit anderen Funktionen, wenn ihr Lust habt, wiederholt werden
- Diese Teilaufgabe kann mit anderen init_fraction - Werten, wenn ihr Lust habt, wiederholt werden
- Diese Teilaufgabe kann mit der anderen Krigingmethode ("simple","ordinary","universal_linear","universal_quadratic") wiederholt werden
- Ihr könnt für die von euch manuell/visuell geschätzte Funktion hier durchlaufen lassen um "euren" Modelfehler besser einordnen zu können 

# Anwendungsbeispiel Wetterdaten (externe Quelle, habe nur copy-pastet)

# Kriging geographical data

Quelle: https://geostat-framework.readthedocs.io/projects/gstools/en/stable/examples/08_geo_coordinates/01_dwd_krige.html#sphx-glr-examples-08-geo-coordinates-01-dwd-krige-py

In this example we are going to interpolate actual temperature data from
the German weather service `DWD <https://www.dwd.de/EN>`_.

Data is retrieved utilizing the beautiful package
`wetterdienst <https://github.com/earthobservations/wetterdienst>`_,
which serves as an API for the DWD data.

For better visualization, we also download a simple shapefile of the German
borderline with `cartopy <https://github.com/SciTools/cartopy>`_.

In order to keep the number of dependecies low, the calls of both functions
shown beneath are commented out.


In [ ]:
border = np.loadtxt("de_borders.txt")
ids, lat, lon, temp = np.loadtxt("2022-06-04.txt").T

First we will estimate the variogram of our temperature data.
As the maximal bin distance we choose 8 degrees, which corresponds to a
chordal length of about 900 km.

Anmerkung: Ich habe daten für einen anderen Tag heruntergeladen, und da sieht das Variogramm einwenig komisch aus. Habe auf 5-Grad eingeschränkt (ähhm, ca. 600km), danach haut das Variogram ab.
Könnt gerne das komplette Variogramm anschauen.

In [ ]:
#bins = gs.standard_bins((lat, lon), max_dist=np.deg2rad(8), latlon=True)
bins = gs.standard_bins((lat, lon), max_dist=np.deg2rad(5), latlon=True)
bin_c, vario = gs.vario_estimate((lat, lon), temp, bins, latlon=True)

Now we can use this estimated variogram to fit a model to it.
Here we will use a :any:`Spherical` model. We select the ``latlon`` option
to use the `Yadrenko` variant of the model to gain a valid model for lat-lon
coordinates and we rescale it to the earth-radius. Otherwise the length
scale would be given in radians representing the great-circle distance.

We deselect the nugget from fitting and plot the result afterwards.

<div class="alert alert-info"><h4>Note</h4><p>You need to plot the Yadrenko variogram, since the standard variogram
   still holds the ordinary routine that is not respecting the great-circle
   distance.
   
   Aha, gut zu wissen :D :D</p></div>

In [ ]:
#Ich habe hier rumgespielt, bis ich das passende Variogram gefunden habe

model = gs.Spherical(latlon=True, rescale=gs.EARTH_RADIUS)
#model = gs.SuperSpherical(latlon=True, rescale=gs.EARTH_RADIUS)
#model = gs.HyperSpherical(latlon=True, rescale=gs.EARTH_RADIUS)
#model = gs.Exponential(latlon=True, rescale=gs.EARTH_RADIUS)
#model = gs.Linear(latlon=True, rescale=gs.EARTH_RADIUS)

#habe nugget auf True gesetzt, sonst passt variogramm nicht
model.fit_variogram(bin_c, vario, nugget=True)
ax = model.plot("vario_yadrenko", x_max=bins[-1]) #fragt mich nicht, was es ist
ax.scatter(bin_c, vario)
print(model)
print("Zugegebenerweise ein schwer zu interpretierendes Variogramm")


Now we want to interpolate the data using :any:`Universal` kriging.
In order to tinker around with the data, we will use a north-south drift
by assuming a linear correlation with the latitude.
This can be done as follows:

In [ ]:
# das ist interessant, was wir bis jetzt in Details nicht angeschaut haben. Hier wird eine Drift-Funktion vorgegben, wenn man den Drift in den Daten erwartet.
# Drift wird bei universal-kriging berücksichtigt (wir haben ja "nur" universal-linear ausgewählt, was im grunde dem entspricht, ohne Vorgabe der Driftfunktion,
# da hat Kriging was lineares reingefittet, ob sinn macht oder nicht
def north_south_drift(lat, lon):
    return lat


uk = gs.krige.Universal(
    model=model,
    cond_pos=(lat, lon),
    cond_val=temp,
    drift_functions=north_south_drift,
)

print("Nur aus reinem Interesse, wie viele Datenpunkte da sind: %d"%len(temp))

Now we generate the kriging field, by defining a lat-lon grid that covers
the whole of Germany. The :any:`Krige` class provides the option to only
krige the mean field, so one can have a glimpse at the estimated drift.

In [ ]:
g_lat = np.arange(47, 56.1, 0.1)
g_lon = np.arange(5, 16.1, 0.1)

uk.set_pos((g_lat, g_lon), mesh_type="structured");
uk(return_var=False, store="temp_field"); #hier die berechnung vom modelfehler wurde deaktiviert
uk(only_mean=True, store="mean_field");

And that's it. Now let's have a look at the generated field and the input data along with the estimated mean:

In [ ]:
#wenn ihr mit modellen spielt, es kann sein, dass höhrere temperaturwerte abgeschnitten werden
#eventuell hier den temperaturbereich anpassen (dient nur der darstellung)
t_min = 12
t_max = 26
levels = np.linspace(t_min, t_max, 64)
fig, ax = plt.subplots(1, 3, figsize=[10, 5], sharey=True)
sca = ax[0].scatter(lon, lat, c=temp, vmin=t_min, vmax=t_max, cmap="coolwarm")
co1 = ax[1].contourf(g_lon, g_lat, uk["temp_field"], levels, cmap="coolwarm")
co2 = ax[2].contourf(g_lon, g_lat, uk["mean_field"], levels, cmap="coolwarm")

[ax[i].plot(border[:, 0], border[:, 1], color="k") for i in range(3)]
[ax[i].set_xlim([5, 16]) for i in range(3)]
[ax[i].set_xlabel("Lon in deg") for i in range(3)]
ax[0].set_ylabel("Lat in deg")

ax[0].set_title("Temperature observations at 2m\nfrom DWD (2020-06-09 12:00)")
ax[1].set_title("Interpolated temperature\nwith North-South drift")
ax[2].set_title("Estimated mean drift\nfrom Universal Kriging")

fmt = dict(orientation="horizontal", shrink=0.5, fraction=0.1, pad=0.2)
fig.colorbar(co2, ax=ax, **fmt).set_label("T in [°C]")

To get a better impression of the estimated north-south drift, we'll take
a look at a cross-section at a longitude of 10 degree:

In [ ]:
fig, ax = plt.subplots()
ax.plot(g_lat, uk["temp_field"][:, 50], label="Interpolated temperature")
ax.plot(g_lat, uk["mean_field"][:, 50], label="North-South mean drift")
ax.set_xlabel("Lat in deg")
ax.set_ylabel("T in [°C]")
ax.set_title("North-South cross-section at 10°")
ax.legend()

Interpretion of the results is now up to you! ;-)


Hübsches Bildchen anzuschauen, man muss aber cartopy installieren, bei mir hat's nicht geklappt

In [ ]:

import cartopy.crs as ccrs

# define a structured field by latitude and longitude
lat = lon = range(-80, 81)
model = gs.Gaussian(latlon=True, len_scale=777, rescale=gs.EARTH_RADIUS)
srf = gs.SRF(model, seed=12345)
field = srf.structured((lat, lon))
# Orthographic plotting with cartopy
ax = plt.subplot(projection=ccrs.Orthographic(-45, 45))
cont = ax.contourf(lon, lat, field, transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_global()
plt.colorbar(cont)